In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import warnings
import tensorflow as tf
from tqdm import tqdm
from glob import glob
from google.colab import drive

import librosa
import librosa.display as dsp
import IPython.display as ipd

warnings.filterwarnings(action='ignore')
drive.mount('/content/drive')

%cd '/content/drive/MyDrive/deep-voice/data/'

Mounted at /content/drive
/content/drive/MyDrive/deep-voice/data


In [2]:
import torch

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu') #GPU 할당

In [3]:
import random

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

seed_everything(813)

In [4]:
# 저장된 데이터 불러오기
X_train_mfcc = np.load('X_train_mfcc.npy')
X_test_mfcc = np.load('X_test_mfcc.npy')
X_val_mfcc = np.load('X_val_mfcc.npy')

y_train = np.load('y_train.npy')
y_test = np.load('y_test.npy')
y_val = np.load('y_val.npy')

## Bi-LSTM

In [13]:
!pip install tensorflow-addons

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 11.9 MB/s eta 0:00:00


In [16]:
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Bidirectional, Dropout, Dense, Input
from tensorflow.keras.models import Sequential
from tensorflow_addons.layers import GroupNormalization

model = Sequential()

model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(100, 1600)))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
# LSTM 레이어 전에 데이터의 차원을 맞추기 위해 필요한 경우 Flatten 레이어 또는 GlobalMaxPooling1D 레이어를 추가할 수 있습니다.
# 이 예시에서는 데이터가 시퀀스이므로, Flatten 대신 Bidirectional LSTM을 직접 사용합니다.
model.add(Bidirectional(LSTM(512)))
model.add(GroupNormalization())
model.add(Dropout(0.8))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'], run_eagerly=True)

# 모델 요약
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 98, 64)            307264    
                                                                 
 max_pooling1d (MaxPooling1  (None, 49, 64)            0         
 D)                                                              
                                                                 
 conv1d_1 (Conv1D)           (None, 47, 128)           24704     
                                                                 
 max_pooling1d_1 (MaxPoolin  (None, 23, 128)           0         
 g1D)                                                            
                                                                 
 bidirectional (Bidirection  (None, 1024)              2625536   
 al)                                                             
                                                        

In [17]:
# 데이터 조정 함수
def adjust_mfcc(mfcc, target_length=1600):
    if mfcc.shape[2] > target_length:
        # 마지막 차원이 target_length보다 길 경우, 처음부터 target_length까지 자름
        adjusted_mfcc = mfcc[:, :, :target_length]
    elif mfcc.shape[2] < target_length:
        # 마지막 차원이 target_length보다 짧은 경우, 0으로 패딩
        padding = np.zeros((mfcc.shape[0], mfcc.shape[1], target_length - mfcc.shape[2]))
        adjusted_mfcc = np.concatenate((mfcc, padding), axis=2)
    else:
        adjusted_mfcc = mfcc
    return adjusted_mfcc

In [18]:
X_train_mfcc_adjusted = adjust_mfcc(X_train_mfcc, 1600)
X_val_mfcc_adjusted = adjust_mfcc(X_val_mfcc, 1600)
X_test_mfcc_adjusted = adjust_mfcc(X_test_mfcc, 1600)

In [19]:
X_train_mfcc_adjusted.shape

(960, 100, 1600)

In [24]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

# ModelCheckpoint 설정
model_checkpoint = ModelCheckpoint(
    'best_model_bi_lstm.h5', # 모델을 저장할 파일 이름
    monitor='val_accuracy', # 모니터링할 대상 ('val_loss'로 설정할 수도 있음)
    mode='max', # 'max'는 정확도를 모니터링할 때, 'min'은 손실을 모니터링할 때 사용
    save_best_only=True # True로 설정하면, 가장 좋은 성능을 보인 모델만 저장
)

# EarlyStopping 콜백 설정
early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1, restore_best_weights=True)

# 모델 학습
history = model.fit(
    X_train_mfcc_adjusted,
    y_train,
    epochs=10,
    validation_data=(X_val_mfcc_adjusted, y_val),
    callbacks=[early_stopping, model_checkpoint] # 콜백 추가
)

Epoch 1/10
30/30 [==============================] - 46s 1s/step - loss: 6.7610e-04 - accuracy: 1.0000 - val_loss: 1.2799e-04 - val_accuracy: 1.0000
Epoch 2/10
30/30 [==============================] - 44s 1s/step - loss: 2.1801e-04 - accuracy: 1.0000 - val_loss: 9.3096e-05 - val_accuracy: 1.0000
Epoch 3/10
30/30 [==============================] - 41s 1s/step - loss: 4.8994e-04 - accuracy: 1.0000 - val_loss: 3.4026e-05 - val_accuracy: 1.0000
Epoch 4/10
30/30 [==============================] - 45s 2s/step - loss: 2.1031e-04 - accuracy: 1.0000 - val_loss: 3.4451e-05 - val_accuracy: 1.0000
Epoch 5/10
30/30 [==============================] - 41s 1s/step - loss: 7.3111e-04 - accuracy: 1.0000 - val_loss: 2.1086e-05 - val_accuracy: 1.0000
Epoch 6/10
30/30 [==============================] - 42s 1s/step - loss: 1.2570e-04 - accuracy: 1.0000 - val_loss: 2.7897e-05 - val_accuracy: 1.0000
Epoch 7/10
30/30 [==============================] - 44s 1s/step - loss: 6.6102e-05 - accuracy: 1.0000 - val_loss

In [26]:
# 모델 불러오기
from tensorflow.keras.models import load_model
best_model = load_model('best_model_bi_lstm.h5')

# 불러온 모델로 테스트 데이터셋 성능 평가
test_loss, test_acc = best_model.evaluate(X_test_mfcc_adjusted, y_test)
print(f'\nTest accuracy: {test_acc}, Test loss: {test_loss}')

10/10 [==============================] - 3s 182ms/step - loss: 0.0016 - accuracy: 1.0000

Test accuracy: 1.0, Test loss: 0.0015965308994054794


In [27]:
import librosa
import numpy as np
from tensorflow.keras.models import load_model

def audio_to_mfcc(audio_path, sr=16000, n_mfcc=100, hop_length=160, n_fft=400):
    # 오디오 파일 로드
    y, sr = librosa.load(audio_path, sr=sr)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc, hop_length=hop_length, n_fft=n_fft)
    return mfcc

def preprocess_mfcc(mfcc, max_length=1600):
    # MFCC 데이터의 길이 조정
    if mfcc.shape[1] < max_length:
        # 길이가 max_length보다 짧은 경우 패딩 추가
        pad_width = max_length - mfcc.shape[1]
        mfcc = np.pad(mfcc, ((0,0), (0,pad_width)), mode='constant')
    else:
        # 길이가 max_length를 초과하는 경우 잘라냄
        mfcc = mfcc[:, :max_length]
    return mfcc

# 모델 로드
best_model = load_model('best_model_bi_lstm.h5')

# 오디오 파일 경로
audio_path = '/content/1-ko-100018_143_1-M-27-100018_143_1_1.wav'

# 오디오 파일을 MFCC로 변환
mfcc = audio_to_mfcc(audio_path)

# MFCC 데이터 길이 조정
mfcc = preprocess_mfcc(mfcc)

# 데이터 전처리
preprocessed_data = np.expand_dims(mfcc, axis=0)  # 배치 차원 추가

# 모델을 사용하여 예측
prediction = best_model.predict(preprocessed_data)

# 예측 결과 출력
if prediction[0] > 0.5:
    print("딥보이스입니다.")
else:
    print("딥보이스가 아닙니다.")

1/1 [==============================] - 1s 964ms/step
딥보이스입니다.


## LSTM

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# MFCC 등의 데이터를 위한 LSTM 모델 정의
# input_shape=(시간 단계 수, 특성 수)

model = Sequential([
    LSTM(50, input_shape=(100, 2970), return_sequences=True),
    LSTM(50),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 모델 요약 출력
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 100, 50)           604200    
                                                                 
 lstm_1 (LSTM)               (None, 50)                20200     
                                                                 
 dense (Dense)               (None, 1)                 51        
                                                                 
Total params: 624451 (2.38 MB)
Trainable params: 624451 (2.38 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

# EarlyStopping 콜백 정의
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=3,
    verbose=1,
    mode='min',
    restore_best_weights=True
)

# 모델 훈련
history = model.fit(
    X_train_mfcc, y_train,
    epochs=10,
    batch_size=32,
    validation_data=(X_val_mfcc, y_val),
    callbacks=[early_stopping]
)

Epoch 1/10
40/40 [==============================] - 24s 470ms/step - loss: 0.1200 - accuracy: 0.9734 - val_loss: 0.0236 - val_accuracy: 0.9937
Epoch 2/10
40/40 [==============================] - 19s 490ms/step - loss: 0.0039 - accuracy: 1.0000 - val_loss: 0.0219 - val_accuracy: 0.9937
Epoch 3/10
40/40 [==============================] - 17s 431ms/step - loss: 0.0017 - accuracy: 1.0000 - val_loss: 0.0229 - val_accuracy: 0.9937
Epoch 4/10
40/40 [==============================] - 19s 480ms/step - loss: 0.0011 - accuracy: 1.0000 - val_loss: 0.0243 - val_accuracy: 0.9937
Epoch 5/10
40/40 [==============================] - 18s 450ms/step - loss: 7.5919e-04 - accuracy: 1.0000 - val_loss: 0.0255 - val_accuracy: 0.9937
Epoch 5: early stopping


## CNN-LSTM

In [ ]:
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Bidirectional, Dropout, Dense, Input
from tensorflow.keras.models import Sequential

model = Sequential()

# Conv1D 레이어 추가. input_shape의 마지막 차원을 2970으로 설정
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(100, 2970)))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))

# LSTM 레이어 추가
model.add(LSTM(50))

# 분류를 위한 완전 연결 레이어 및 출력 레이어를 수정합니다.
model.add(Dense(1, activation='sigmoid'))

# 모델 컴파일
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

# EarlyStopping 콜백 정의
early_stopping = EarlyStopping(
    monitor='val_loss',  # 관찰할 값
    patience=3,          # 성능 향상이 없는 에폭을 몇 번이나 허용할 것인가
    verbose=1,           # 진행 상황을 출력
    mode='min',          # 'min' 모드는 관찰하는 값의 감소가 멈추면 훈련을 중단
    restore_best_weights=True  # 가장 좋은 모델의 가중치를 복원
)

# 모델 훈련
history = model.fit(
    X_train_mfcc, y_train,
    epochs=10,
    batch_size=32,
    validation_data=(X_val_mfcc, y_val),
    callbacks=[early_stopping]  # 콜백 리스트에 early_stopping 추가
)

Epoch 1/10
40/40 [==============================] - 17s 353ms/step - loss: 0.0902 - accuracy: 0.9773 - val_loss: 0.0090 - val_accuracy: 0.9969
Epoch 2/10
40/40 [==============================] - 12s 310ms/step - loss: 0.0044 - accuracy: 1.0000 - val_loss: 0.0050 - val_accuracy: 1.0000
Epoch 3/10
40/40 [==============================] - 14s 342ms/step - loss: 0.0031 - accuracy: 1.0000 - val_loss: 0.0031 - val_accuracy: 1.0000
Epoch 4/10
40/40 [==============================] - 13s 330ms/step - loss: 0.0023 - accuracy: 1.0000 - val_loss: 0.0025 - val_accuracy: 1.0000
Epoch 5/10
40/40 [==============================] - 13s 335ms/step - loss: 0.0018 - accuracy: 1.0000 - val_loss: 0.0020 - val_accuracy: 1.0000
Epoch 6/10
40/40 [==============================] - 13s 335ms/step - loss: 0.0014 - accuracy: 1.0000 - val_loss: 0.0016 - val_accuracy: 1.0000
Epoch 7/10
40/40 [==============================] - 13s 334ms/step - loss: 0.0012 - accuracy: 1.0000 - val_loss: 0.0013 - val_accuracy: 1.0000

## CNN-BiLSTM

In [ ]:
!pip install tensorflow-addons

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 5.0 MB/s eta 0:00:00


In [ ]:
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Bidirectional, Dropout, Dense, Input
from tensorflow.keras.models import Sequential
from tensorflow_addons.layers import GroupNormalization

model = Sequential()

# Conv1D 레이어 추가. input_shape의 마지막 차원을 2970으로 설정
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(100, 2970)))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
# LSTM 레이어 전에 데이터의 차원을 맞추기 위해 필요한 경우 Flatten 레이어 또는 GlobalMaxPooling1D 레이어를 추가할 수 있습니다.
# 이 예시에서는 데이터가 시퀀스이므로, Flatten 대신 Bidirectional LSTM을 직접 사용합니다.
model.add(Bidirectional(LSTM(512)))
model.add(GroupNormalization())
model.add(Dropout(0.8))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'], run_eagerly=True)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

# EarlyStopping 콜백 정의
early_stopping = EarlyStopping(
    monitor='val_loss',  # 관찰할 값
    patience=3,          # 성능 향상이 없는 에폭을 몇 번이나 허용할 것인가
    verbose=1,           # 진행 상황을 출력
    mode='min',          # 'min' 모드는 관찰하는 값의 감소가 멈추면 훈련을 중단
    restore_best_weights=True  # 가장 좋은 모델의 가중치를 복원
)

# 모델 훈련
history = model.fit(
    X_train_mfcc, y_train,
    epochs=10,
    batch_size=32,
    validation_data=(X_val_mfcc, y_val),
    callbacks=[early_stopping]  # 콜백 리스트에 early_stopping 추가
)

Epoch 1/10
40/40 [==============================] - 57s 1s/step - loss: 0.1352 - accuracy: 0.9477 - val_loss: 0.0015 - val_accuracy: 1.0000
Epoch 2/10
40/40 [==============================] - 53s 1s/step - loss: 3.4366e-04 - accuracy: 1.0000 - val_loss: 0.0028 - val_accuracy: 0.9969
Epoch 3/10
40/40 [==============================] - 55s 1s/step - loss: 2.4034e-04 - accuracy: 1.0000 - val_loss: 0.0020 - val_accuracy: 1.0000
Epoch 4/10
40/40 [==============================] - 55s 1s/step - loss: 6.9047e-04 - accuracy: 1.0000 - val_loss: 0.0065 - val_accuracy: 0.9969
Epoch 4: early stopping


## RNN

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense

# RNN 모델 정의
model = Sequential([
    SimpleRNN(50, input_shape=(100, 2970), return_sequences=True),
    SimpleRNN(50),
    # 10개의 출력 클래스에 대한 Dense 레이어. 분류를 위해 softmax 활성화 함수를 사용합니다.
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# 모델 요약
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn (SimpleRNN)      (None, 100, 50)           151050    
                                                                 
 simple_rnn_1 (SimpleRNN)    (None, 50)                5050      
                                                                 
 dense_2 (Dense)             (None, 1)                 51        
                                                                 
Total params: 156151 (609.96 KB)
Trainable params: 156151 (609.96 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

# EarlyStopping 콜백 정의
early_stopping = EarlyStopping(
    monitor='val_loss',  # 관찰할 값
    patience=3,          # 성능 향상이 없는 에폭을 몇 번이나 허용할 것인가
    verbose=1,           # 진행 상황을 출력
    mode='min',          # 'min' 모드는 관찰하는 값의 감소가 멈추면 훈련을 중단
    restore_best_weights=True  # 가장 좋은 모델의 가중치를 복원
)

# 모델 훈련
history = model.fit(
    X_train_mfcc, y_train,
    epochs=10,
    batch_size=32,
    validation_data=(X_val_mfcc, y_val),
    callbacks=[early_stopping]  # 콜백 리스트에 early_stopping 추가
)

Epoch 1/10
40/40 [==============================] - 12s 247ms/step - loss: 0.2032 - accuracy: 0.9102 - val_loss: 0.0443 - val_accuracy: 0.9937
Epoch 2/10
40/40 [==============================] - 8s 195ms/step - loss: 0.0093 - accuracy: 1.0000 - val_loss: 0.0346 - val_accuracy: 0.9937
Epoch 3/10
40/40 [==============================] - 10s 263ms/step - loss: 0.0043 - accuracy: 1.0000 - val_loss: 0.0336 - val_accuracy: 0.9937
Epoch 4/10
40/40 [==============================] - 12s 304ms/step - loss: 0.0028 - accuracy: 1.0000 - val_loss: 0.0339 - val_accuracy: 0.9937
Epoch 5/10
40/40 [==============================] - 12s 297ms/step - loss: 0.0020 - accuracy: 1.0000 - val_loss: 0.0344 - val_accuracy: 0.9937
Epoch 6/10
40/40 [==============================] - 8s 186ms/step - loss: 0.0015 - accuracy: 1.0000 - val_loss: 0.0351 - val_accuracy: 0.9937
Epoch 6: early stopping


## GRU

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense

# 모델 정의
model = Sequential([
    GRU(50, input_shape=(100, 2970), return_sequences=True),  # LSTM 대신 GRU 사용
    GRU(50),  # 여기도 동일하게 GRU 사용
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 모델 요약
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru (GRU)                   (None, 100, 50)           453300    
                                                                 
 gru_1 (GRU)                 (None, 50)                15300     
                                                                 
 dense_3 (Dense)             (None, 1)                 51        
                                                                 
Total params: 468651 (1.79 MB)
Trainable params: 468651 (1.79 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

# EarlyStopping 콜백 정의
early_stopping = EarlyStopping(
    monitor='val_loss',  # 관찰할 값
    patience=3,          # 성능 향상이 없는 에폭을 몇 번이나 허용할 것인가
    verbose=1,           # 진행 상황을 출력
    mode='min',          # 'min' 모드는 관찰하는 값의 감소가 멈추면 훈련을 중단
    restore_best_weights=True  # 가장 좋은 모델의 가중치를 복원
)

# 모델 훈련
history = model.fit(
    X_train_mfcc, y_train,
    epochs=10,
    batch_size=32,
    validation_data=(X_val_mfcc, y_val),
    callbacks=[early_stopping]  # 콜백 리스트에 early_stopping 추가
)

Epoch 1/10
40/40 [==============================] - 22s 425ms/step - loss: 0.1069 - accuracy: 0.9516 - val_loss: 0.0181 - val_accuracy: 0.9969
Epoch 2/10
40/40 [==============================] - 16s 403ms/step - loss: 0.0016 - accuracy: 1.0000 - val_loss: 0.0185 - val_accuracy: 0.9969
Epoch 3/10
40/40 [==============================] - 15s 385ms/step - loss: 9.0624e-04 - accuracy: 1.0000 - val_loss: 0.0185 - val_accuracy: 0.9969
Epoch 4/10
40/40 [==============================] - 16s 401ms/step - loss: 6.4945e-04 - accuracy: 1.0000 - val_loss: 0.0187 - val_accuracy: 0.9969
Epoch 4: early stopping


## Bi-GRU

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, GRU, Dense

# 모델 정의
model = Sequential([
    Bidirectional(GRU(50, return_sequences=True), input_shape=(100, 2970)),  # 첫 번째 GRU 레이어를 양방향으로 설정
    Bidirectional(GRU(50)),  # 두 번째 GRU 레이어도 양방향으로 설정
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 모델 요약
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional (Bidirection  (None, 100, 100)          906600    
 al)                                                             
                                                                 
 bidirectional_1 (Bidirecti  (None, 100)               45600     
 onal)                                                           
                                                                 
 dense_4 (Dense)             (None, 1)                 101       
                                                                 
Total params: 952301 (3.63 MB)
Trainable params: 952301 (3.63 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

# EarlyStopping 콜백 정의
early_stopping = EarlyStopping(
    monitor='val_loss',  # 관찰할 값
    patience=3,          # 성능 향상이 없는 에폭을 몇 번이나 허용할 것인가
    verbose=1,           # 진행 상황을 출력
    mode='min',          # 'min' 모드는 관찰하는 값의 감소가 멈추면 훈련을 중단
    restore_best_weights=True  # 가장 좋은 모델의 가중치를 복원
)

# 모델 훈련
history = model.fit(
    X_train_mfcc, y_train,
    epochs=10,
    batch_size=32,
    validation_data=(X_val_mfcc, y_val),
    callbacks=[early_stopping]  # 콜백 리스트에 early_stopping 추가
)

Epoch 1/10
40/40 [==============================] - 39s 824ms/step - loss: 0.1068 - accuracy: 0.9633 - val_loss: 0.0086 - val_accuracy: 0.9969
Epoch 2/10
40/40 [==============================] - 31s 772ms/step - loss: 0.0016 - accuracy: 1.0000 - val_loss: 0.0061 - val_accuracy: 0.9969
Epoch 3/10
40/40 [==============================] - 30s 761ms/step - loss: 6.3716e-04 - accuracy: 1.0000 - val_loss: 0.0060 - val_accuracy: 0.9969
Epoch 4/10
40/40 [==============================] - 32s 809ms/step - loss: 4.1769e-04 - accuracy: 1.0000 - val_loss: 0.0057 - val_accuracy: 0.9969
Epoch 5/10
40/40 [==============================] - 32s 795ms/step - loss: 2.9768e-04 - accuracy: 1.0000 - val_loss: 0.0056 - val_accuracy: 0.9969
Epoch 6/10
40/40 [==============================] - 32s 809ms/step - loss: 2.2589e-04 - accuracy: 1.0000 - val_loss: 0.0054 - val_accuracy: 0.9969
Epoch 7/10
40/40 [==============================] - 32s 786ms/step - loss: 1.7898e-04 - accuracy: 1.0000 - val_loss: 0.0053 - 